# Application: new experiment

This notebook allows users to find specific genes in their experiment of interest using an existing VAE model

This notebook will generate a `generic_gene_summary_<experiment id>.tsv` file that contains a z-score per gene that indicates how specific a gene is the experiment in question.

In [1]:
%load_ext autoreload
%load_ext rpy2.ipython
%autoreload 2

In [2]:
import os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from ponyo import utils
from generic_expression_patterns_modules import (
    process,
    new_experiment_process,
    stats,
    ranking,
)

/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
Using TensorFlow backend.


## User input

User needs to define the following in the [config file](../configs/config_new_experiment.tsv):

1. Template experiment (`raw_template_filename`). This is the experiment you are interested in studying.
2. Training compendium used to train VAE, including unnormalized gene mapped version (`mapped_compendium_filename`) and normalized version (`normalized_compendium_filename`). Links to these datasets can be found in the README.
3. Scaler transform (`scaler_filename`) used to normalize the training compendium. This can be found in the `data/` directory within the analysis folder.
4. Directory (`vae_model_dir`) containing trained VAE model.
5. Experiment id (`project_id`) to label newly create simulated experiments.
6. The number of experiments to simulate (`num_simulated`)
7. Minimum average read count to filter data by (`count_threshold`)
8. Size of existing models latent dimension (`latent_dim`). Should be 30.
9. Name of column header (`rank_genes_by`) from DE association statistic results. This column will be use to rank genes. Select "logFC", "P.Value", "adj.P.Val", "t" if using Limma. Select "log2FoldChange", "pvalue", "padj" if using DESeq.
10. `DE_logFC_name` is either "logFC" or "log2FoldChange". This is used for plotting volcano plots.
11. `DE_pvalue_name` is either "adj.P.Val" or "padj". This is used for plotting volcano plots.

The remaining parameters specify filenames that intermediate data files will be written to.

The user also needs to provide metadata files:
1. `<experiment id>_process_samples.tsv` contains 2 columns (sample ids, label that indicates if the sample is kept or removed). See [example](data/metadata/cis-gem-par-KU1919_process_samples.tsv). **Note: This file is not required if the user wishes to use all the samples in the template experiment file.**
2. `<experiment id>_groups.tsv` contains 2 columns: sample ids, group label to perform DE analysis. See [example](data/metadata/cis-gem-par-KU1919_groups.tsv)

In [3]:
# Read in config variables
base_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))

config_filename = os.path.abspath(
    os.path.join(base_dir, "configs", "config_new_experiment.tsv")
)

params = utils.read_config(config_filename)

In [4]:
# Load config params

# Local directory to store intermediate files
local_dir = params["local_dir"]

# Number of simulated experiments to generate
num_runs = params["num_simulated"]

# Directory containing trained VAE model
vae_model_dir = params["vae_model_dir"]

# Dimension of latent space used in VAE model
latent_dim = params["latent_dim"]

# ID for template experiment
# This ID will be used to label new simulated experiments
project_id = params["project_id"]

# Template experiment filename
template_filename = params["raw_template_filename"]
mapped_template_filename = params["mapped_template_filename"]
normalized_template_filename = params["normalized_template_filename"]
processed_template_filename = params["processed_template_filename"]

# Training dataset used for existing VAE model
mapped_compendium_filename = params["mapped_compendium_filename"]

# Normalized compendium filename
normalized_compendium_filename = params["normalized_compendium_filename"]

# Scaler transform used to scale compendium data into 0-1 range for training
scaler_filename = params["scaler_filename"]

# Test statistic used to rank genes by
col_to_rank_genes = params["rank_genes_by"]

# Minimum mean count per gene
count_threshold = params["count_threshold"]

# Column headers to use to make summary statistic table
logFC_name = params["DE_logFC_name"]
pvalue_name = params["DE_pvalue_name"]

In [5]:
# Load metadata files

# Load metadata file with processing information
sample_id_metadata_filename = os.path.join(
    "data", "metadata", f"{project_id}_process_samples.tsv"
)

# Load metadata file with grouping assignments for samples
metadata_filename = os.path.join("data", "metadata", f"{project_id}_groups.tsv")

In [6]:
# Output filename
gene_summary_filename = f"generic_gene_summary_{project_id}.tsv"

## Map template experiment to same feature space as training compendium

In order to simulate a new gene expression experiment, we will need to encode this experiment into the learned latent space. This requires that the feature space (i.e. genes) in the template experiment match the features in the compendium used to train the VAE model. These cells process the template experiment to be of the expected input format:
* Template data is expected to be a matrix that is sample x gene
* Template experiment is expected to have the same genes as the compendium experiment. Genes that are in the template experiment but not in the compendium are removed. Genes that are in the compendium but missing in the template experiment are added and the gene expression value is set to the median gene expression value of that gene across the samples in the compendium.

In [7]:
# Template experiment needs to be of the form sample x gene
template_filename_only = template_filename.split("/")[-1].split(".")[0]
transposed_template_filename = os.path.join(
    local_dir, template_filename_only + "_transposed.txt"
)

new_experiment_process.transpose_save(template_filename, transposed_template_filename)

In [8]:
new_experiment_process.process_template_experiment(
    transposed_template_filename,
    mapped_compendium_filename,
    scaler_filename,
    mapped_template_filename,
    normalized_template_filename,
)

(72, 58528)
(49651, 17755)


## Simulate experiments based on template experiment

Embed template experiment into learned latent space and linearly shift template experiment to different locations of the latent space to create new experiments

In [9]:
# Simulate experiments based on template experiment
normalized_compendium_data = pd.read_csv(
    normalized_compendium_filename, sep="\t", index_col=0, header=0
)
normalized_template_data = pd.read_csv(
    normalized_template_filename, sep="\t", index_col=0, header=0
)

for run_id in range(num_runs):
    new_experiment_process.embed_shift_template_experiment(
        normalized_compendium_data,
        normalized_template_data,
        vae_model_dir,
        project_id,
        scaler_filename,
        local_dir,
        latent_dim,
        run_id,
    )

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



## Process template and simulated experiments

* Remove samples not required for comparison
* Make sure ordering of samples matches metadata for proper comparison
* Make sure values are cast as integers if using DESeq
* Filter lowly expressed genes if using DESeq

In [10]:
if "human_general_analysis" in vae_model_dir:
    method = "deseq"
else:
    method = "limma"

In [11]:
if not os.path.exists(sample_id_metadata_filename):
    sample_id_metadata_filename = None

if method == "deseq":
    stats.process_samples_for_DESeq(
        mapped_template_filename,
        metadata_filename,
        processed_template_filename,
        count_threshold,
        sample_id_metadata_filename,
    )

    for i in range(num_runs):
        simulated_filename = os.path.join(
            local_dir,
            "pseudo_experiment",
            f"selected_simulated_data_{project_id}_{i}.txt",
        )
        out_simulated_filename = os.path.join(
            local_dir,
            "pseudo_experiment",
            f"selected_simulated_data_{project_id}_{i}_processed.txt",
        )
        stats.process_samples_for_DESeq(
            simulated_filename,
            metadata_filename,
            out_simulated_filename,
            count_threshold,
            sample_id_metadata_filename,
        )
else:
    stats.process_samples_for_limma(
        mapped_template_filename,
        metadata_filename,
        processed_template_filename,
        sample_id_metadata_filename,
    )

    for i in range(num_runs):
        simulated_filename = os.path.join(
            local_dir,
            "pseudo_experiment",
            f"selected_simulated_data_{project_id}_{i}.txt",
        )
        stats.process_samples_for_limma(
            simulated_filename,
            metadata_filename,
            None,
            sample_id_metadata_filename,
        )

sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly


## Differential expression analysis

* If data is RNA-seq then use DESeq2 (using human_general_analysis model)
* If data is microarray then use Limma (using human_cancer_analysis, pseudomonas_analysis models)

In [12]:
# Create subdirectory: "<local_dir>/DE_stats/"
os.makedirs(os.path.join(local_dir, "DE_stats"), exist_ok=True)

In [13]:
%%R -i metadata_filename -i project_id -i processed_template_filename -i local_dir -i base_dir -i method

source(paste0(base_dir, '/generic_expression_patterns_modules/DE_analysis.R'))

# File created: "<local_dir>/DE_stats/DE_stats_template_data_<project_id>_real.txt"
if (method == "deseq"){
    get_DE_stats_DESeq(
        metadata_filename,
        project_id,
        processed_template_filename,
        "template",
        local_dir,
        "real"
    )
}
else{
    get_DE_stats_limma(
        metadata_filename,
        project_id,
        processed_template_filename,
        "template",
        local_dir,
        "real"
    )
}

/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Loading required package: S4Vectors

  warnings.warn(x, RRuntimeWarning)
/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Loading required package: stats4

  warnings.warn(x, RRuntimeWarning)
/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Loading required package: BiocGenerics

  warnings.warn(x, RRuntimeWarning)
/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Loading required package: parallel

  warnings.warn(x, RRuntimeWarning)
/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 
Attaching package: ‘BiocGenerics’


  warnings.warn(x, RRuntimeWarni

[1] "Checking sample ordering..."
[1] TRUE


In [14]:
%%R -i metadata_filename -i project_id -i base_dir -i local_dir -i num_runs -i method

source(paste0(base_dir, '/generic_expression_patterns_modules/DE_analysis.R'))

# Files created: "<local_dir>/DE_stats/DE_stats_simulated_data_<project_id>_<n>.txt"
for (i in 0:(num_runs-1)){
    simulated_data_filename <- paste(
        local_dir,
        "pseudo_experiment/selected_simulated_data_",
        project_id,
        "_",
        i,
        "_processed.txt",
        sep = ""
    )
    if (method == "deseq"){
        get_DE_stats_DESeq(
            metadata_filename,
            project_id,
            simulated_data_filename,
            "simulated",
            local_dir,
            i
            )
    }
    else {
        get_DE_stats_limma(
            metadata_filename,
            project_id,
            simulated_data_filename,
            "simulated",
            local_dir,
            i
            )
        }
    }

/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: -- replacing outliers and refitting for 74 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

  warnings.warn(x, RRuntimeWarning)
/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: -- replacing outliers and refitting for 220 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

  warnings.warn(x, RRuntimeWarning)
/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: -- replacing outliers and refitting for 1837 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

  warnings.warn(x, RRuntimeWarning)
/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/

[1] "Checking sample ordering..."
[1] TRUE
[1] "Checking sample ordering..."
[1] TRUE
[1] "Checking sample ordering..."
[1] TRUE
[1] "Checking sample ordering..."
[1] TRUE
[1] "Checking sample ordering..."
[1] TRUE
[1] "Checking sample ordering..."
[1] TRUE
[1] "Checking sample ordering..."
[1] TRUE
[1] "Checking sample ordering..."
[1] TRUE
[1] "Checking sample ordering..."
[1] TRUE
[1] "Checking sample ordering..."
[1] TRUE
[1] "Checking sample ordering..."
[1] TRUE
[1] "Checking sample ordering..."
[1] TRUE
[1] "Checking sample ordering..."
[1] TRUE
[1] "Checking sample ordering..."
[1] TRUE
[1] "Checking sample ordering..."
[1] TRUE
[1] "Checking sample ordering..."
[1] TRUE
[1] "Checking sample ordering..."
[1] TRUE
[1] "Checking sample ordering..."
[1] TRUE
[1] "Checking sample ordering..."
[1] TRUE
[1] "Checking sample ordering..."
[1] TRUE
[1] "Checking sample ordering..."
[1] TRUE
[1] "Checking sample ordering..."
[1] TRUE
[1] "Checking sample ordering..."
[1] TRUE
[1] "Checki

## Rank genes

Genes are ranked by their "generic-ness" - how frequently these genes are changed across the simulated experiments using user-specific test statistic (i.e. log2 fold change).

In [15]:
analysis_type = "DE"
template_DE_stats_filename = os.path.join(
    local_dir, "DE_stats", f"DE_stats_template_data_{project_id}_real.txt"
)

template_DE_stats, simulated_DE_summary_stats = ranking.process_and_rank_genes_pathways(
    template_DE_stats_filename,
    local_dir,
    num_runs,
    project_id,
    analysis_type,
    col_to_rank_genes,
    logFC_name,
    pvalue_name,
)

## Summary table

* Gene ID: Gene identifier (hgnc symbols for human data or PA number for *P. aeruginosa* data)
* (Real): Statistics for template experiment
* (Simulated): Statistics across simulated experiments
* Number of experiments: Number of simulated experiments
* Z-score: High z-score indicates that gene is more changed in template compared to the null set of simulated experiments (high z-score = highly specific to template experiment)


**Note:**
* If using DESeq, genes with NaN in only the `Adj P-value (Real)` column are those genes flagged because of the `cooksCutoff` parameter. The cook's distance as a diagnostic to tell if a single sample has a count which has a disproportionate impact on the log fold change and p-values. These genes are flagged with an NA in the pvalue and padj columns of the result table.

* If using DESeq with count threshold, some genes may not be present in all simulated experiments (i.e. the `Number of experiments (simulated)` will not equal the number of simulated experiments you specified in the beginning. This pre-filtering will lead to some genes found in few simulated experiments and so the background/null set for that gene is not robust. Thus, the user should sort by both z-score and number of experiments to identify specific expressed genes.

* If using DESeq without count threshold, some genes may still not be present in all simulated experiments (i.e. the `Number of experiments (simulated)`  will not equal the number of simulated experiments you specified in the beginning. If the gene is 0 expressed across all samples and thus automatically given an NA in `log fold change, adjusted p-value` columns. Thus, the user should sort by both z-score and number of experiments to identify specific expressed genes.

For more information you can read [DESeq FAQs](https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#pvaluesNA)

In [16]:
# Get summary table
summary_gene_ranks = ranking.generate_summary_table(
    template_DE_stats_filename,
    template_DE_stats,
    simulated_DE_summary_stats,
    col_to_rank_genes,
    local_dir,
    "gene",
    params,
)

summary_gene_ranks.sort_values(by="Z score", ascending=False).head(10)

/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


,Gene ID,Adj P-value (Real),Rank (Real),abs(log2FoldChange) (Real),log2FoldChange (Real),Median adj p-value (simulated),Rank (simulated),Mean abs(log2FoldChange) (simulated),Std deviation (simulated),Number of experiments (simulated),Z score
RXFP2,RXFP2,2.039775e-09,17159.0,21.707360,-21.707360,0.999885,11427.0,0.404230,0.406799,24,52.367643
GPX7,GPX7,6.607610e-03,17153.0,6.082224,6.082224,0.999880,12541.0,0.318692,0.218875,25,26.332482
LRRC4,LRRC4,2.994721e-05,17151.0,5.863040,-5.863040,0.999841,12175.0,0.342337,0.229571,25,24.047880
DCN,DCN,1.660303e-06,17158.0,8.829010,-8.829010,0.999889,6224.0,0.316071,0.408809,25,20.823779
CDH11,CDH11,3.358492e-02,17154.0,6.143647,-6.143647,0.999880,8694.0,0.342905,0.358618,25,16.175274
EYA4,EYA4,3.603901e-03,17140.0,5.136665,-5.136665,0.999889,11319.0,0.338946,0.310174,25,15.467823
RRM1,RRM1,8.404496e-08,16913.0,2.990056,2.990056,0.999880,10521.0,0.263583,0.177253,25,15.381820
ZNF366,ZNF366,5.178133e-02,16996.0,3.428502,3.428502,0.999889,10878.0,0.278472,0.208072,25,15.139151
HTRA3,HTRA3,2.245523e-02,17016.0,3.489279,-3.489279,0.999889,2486.0,0.187404,0.218743,25,15.094767
XPC,XPC,3.529001e-02,13369.0,0.577698,0.577698,0.999889,135.0,0.069185,0.035642,25,14.267395


In [17]:
summary_gene_ranks.isna().any()

Gene ID                                 False
Adj P-value (Real)                       True
Rank (Real)                              True
abs(log2FoldChange) (Real)               True
log2FoldChange (Real)                    True
Median adj p-value (simulated)          False
Rank (simulated)                        False
Mean abs(log2FoldChange) (simulated)    False
Std deviation (simulated)               False
Number of experiments (simulated)       False
Z score                                  True
dtype: bool

In [18]:
summary_gene_ranks[summary_gene_ranks.isna().any(axis=1)]

,Gene ID,Adj P-value (Real),Rank (Real),abs(log2FoldChange) (Real),log2FoldChange (Real),Median adj p-value (simulated),Rank (simulated),Mean abs(log2FoldChange) (simulated),Std deviation (simulated),Number of experiments (simulated),Z score
ACOD1,ACOD1,NaN,NaN,NaN,NaN,0.999889,13655.0,0.758958,0.925459,25,NaN
ACTL9,ACTL9,NaN,NaN,NaN,NaN,0.999885,17555.0,0.955850,0.769786,24,NaN
ACTRT2,ACTRT2,NaN,NaN,NaN,NaN,0.999904,17524.0,0.908983,0.582880,24,NaN
ADAM30,ADAM30,NaN,NaN,NaN,NaN,0.999885,14966.0,0.632421,0.651123,24,NaN
ADAM6,ADAM6,NaN,NaN,NaN,NaN,0.999904,16311.0,0.659662,0.597932,24,NaN
ADAMDEC1,ADAMDEC1,NaN,NaN,NaN,NaN,0.999880,15939.0,0.786723,1.042967,25,NaN
AKAIN1,AKAIN1,NaN,NaN,NaN,NaN,0.999833,17533.0,0.966889,0.747688,24,NaN
ALPI,ALPI,NaN,NaN,NaN,NaN,0.999880,16805.0,1.008875,1.346828,25,NaN
AMELY,AMELY,NaN,NaN,NaN,NaN,0.999889,15012.0,0.476614,0.419563,23,NaN
ANHX,ANHX,NaN,NaN,NaN,NaN,0.999904,16978.0,0.846781,1.046244,24,NaN


In [19]:
# Save
summary_gene_ranks.to_csv(gene_summary_filename, sep="\t")